In [46]:
import torch # 딥러닝 프레임워크 파이토치 불러오기
import torch.nn as nn # 다양한 레이어, 활성화 함수,
import torch.nn.functional as F # F: PyTorch의 torch.nn.functional 모듈의 약칭
import torch.optim as optim # 최적화 알고리즘들이 담겨있는 모듈

In [47]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [48]:
bitcoin = [82043000, 81351000, 83825000, 181939]
solana = [196800, 196300, 205800, 181939]
dogicoin = [152, 150, 156, 181939]
ripple = [610, 608, 622, 181939]
pepecoin = [0.02, 0.02, 0.02, 181939]
x_train = torch.FloatTensor([bitcoin,
                             solana,
                             dogicoin,
                             ripple,
                             pepecoin,])
y_train = torch.FloatTensor([[82601000],
                              [202200],
                              [153],
                              [621],
                              [0.02],])

In [49]:
print(x_train)
print(x_train.shape)
print(y_train.shape)

tensor([[8.2043e+07, 8.1351e+07, 8.3825e+07, 1.8194e+05],
        [1.9680e+05, 1.9630e+05, 2.0580e+05, 1.8194e+05],
        [1.5200e+02, 1.5000e+02, 1.5600e+02, 1.8194e+05],
        [6.1000e+02, 6.0800e+02, 6.2200e+02, 1.8194e+05],
        [2.0000e-02, 2.0000e-02, 2.0000e-02, 1.8194e+05]])
torch.Size([5, 4])
torch.Size([5, 1])


In [50]:
from sklearn.preprocessing import RobustScaler

rs = RobustScaler()

x_train_scaled = rs.fit_transform(x_train)
y_train_scaled = rs.fit_transform(y_train)

x_train_scaled = torch.FloatTensor(x_train_scaled)
y_train_scaled = torch.FloatTensor(y_train_scaled)

print(x_train_scaled)
print(y_train_scaled)

tensor([[ 4.1720e+02,  4.1474e+02,  4.0762e+02,  0.0000e+00],
        [ 9.9767e-01,  9.9767e-01,  9.9773e-01,  0.0000e+00],
        [-2.3290e-03, -2.3349e-03, -2.2661e-03,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.1019e-03, -3.0996e-03, -3.0245e-03,  0.0000e+00]])
tensor([[ 4.0882e+02],
        [ 9.9768e-01],
        [-2.3163e-03],
        [ 0.0000e+00],
        [-3.0734e-03]])


In [58]:
dataset = TensorDataset(x_train_scaled, y_train_scaled) # x_train_scaled, y_train_scaled을 하나의 데이터셋으로 만듦

In [59]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True,  drop_last=False) # data loader 데이터셋을 반복 가능한 형태로 만듦 / drop_last=False: 마지막 배치를 버리지 않음

In [56]:
model = nn.Linear(4,1) # 입력 텐서에 대해 선형 변환 수행 / y= w1x1 + w2x2 + w3x3 + w4x4 + b / 4는 입력텐서가 4개의 feature를 가지며, 1은 출력텐서가 1개의 출력 feature를 가짐

In [54]:
print(list(model.parameters())) # 4개의 weight와 1개의 b가 들어있음을 확인

[Parameter containing:
tensor([[ 0.4548, -0.4999, -0.4278, -0.1634]], requires_grad=True), Parameter containing:
tensor([0.1316], requires_grad=True)]


In [57]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) # 모델 파라미터에 대해 확률적 경사하강법과 learning rate 적용

In [55]:
epochs = 5
for epoch in range(epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    optimizer.zero_grad() # 기울기 초기화

    x_train_scaled, y_train_scaled = samples # data loader에서 받아온 x, y train 값을 samples에서 할당
    pred = model(x_train_scaled) # 모델 예측

    print(x_train_scaled)
    print(y_train_scaled)

    cost = F.mse_loss(pred, y_train_scaled) # 예측 값과 실제 값 사이의 평균 제곱 오차를 계산

    cost.backward() # 비용 함수를 미분하여 gradient 계산
    optimizer.step() # W와 b를 업데이트

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, epochs, batch_idx+1, len(dataloader),
        cost.item()
        )) # Epoch 전체 에포크 수 표시, Batch 배치 번호 표시

tensor([[  0.0000,   0.0000,   0.0000,   0.0000],
        [417.2043, 414.7356, 407.6189,   0.0000]])
tensor([[  0.0000],
        [408.8177]])
Epoch    0/5 Batch 1/3 Cost: 180378.546875
tensor([[-0.0023, -0.0023, -0.0023,  0.0000],
        [-0.0031, -0.0031, -0.0030,  0.0000]])
tensor([[-0.0023],
        [-0.0031]])
Epoch    0/5 Batch 2/3 Cost: 0.014772
tensor([[0.9977, 0.9977, 0.9977, 0.0000]])
tensor([[0.9977]])
Epoch    0/5 Batch 3/3 Cost: 37.162655
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0023, -0.0023, -0.0023,  0.0000]])
tensor([[ 0.0000],
        [-0.0023]])
Epoch    1/5 Batch 1/3 Cost: 0.017102
tensor([[ 0.9977,  0.9977,  0.9977,  0.0000],
        [-0.0031, -0.0031, -0.0030,  0.0000]])
tensor([[ 0.9977],
        [-0.0031]])
Epoch    1/5 Batch 2/3 Cost: 18.585440
tensor([[417.2043, 414.7356, 407.6189,   0.0000]])
tensor([[408.8177]])
Epoch    1/5 Batch 3/3 Cost: 6129033.500000
tensor([[-0.0023, -0.0023, -0.0023,  0.0000],
        [-0.0031, -0.0031, -0.0030,  0.00